In [84]:
import pandas as pd
import os
from glob import glob
import json

In [85]:
entries = []

for onnx_path in glob("/ssd1/robustness_onnx_zoo/*/*.onnx"):
    directory, file = os.path.split(onnx_path)
    dataset = "imagenet"
    if "_cifar10_" in file:
        dataset = "cifar10"
    elif "_cifar100_" in file:
        dataset = "cifar100"   
        
    robust = True
    if "hso_normal_training_" in file or "timm_" in file:
        robust = False
        
    paper_id = file.replace(".onnx", "").replace("hso_normal_training_", "").replace("robustbench_", "") \
                        .replace("timm_", "").replace("_imagenet1k_13", "").replace("_cifar10_13", "").replace("_cifar100_13", "") \
                        .replace("_imagenet_linf_13", "").replace("_cifar10_linf_13", "").replace("_cifar100_linf_13", "")
    
    
    if paper_id.startswith("Standard"):
        continue
    
    paper_link = None
    clean_acc = 0
    robust_acc = 0
    architecture = None
    additional_data = False
    
    if not file.startswith("timm_"): 
        with open(f"./robustbench/model_info/{dataset}/Linf/{paper_id}.json", "rb") as json_stream:
            model_info = json.load(json_stream)
            paper_link = model_info["link"]
            name = model_info["name"]
            clean_acc = model_info["clean_acc"]
            robust_acc = model_info["autoattack_acc"]
            architecture = model_info["architecture"]
            additional_data = model_info["additional_data"]
            
    if file.startswith("hso_normal_training_"):
        clean_acc = 0
        robust_acc = 0
        additional_data = False
        benchmark_log_path = f"./robust_benchmark/{dataset}/robustbench_{paper_id}_{dataset}.log"
        if os.path.exists(benchmark_log_path):
            with open(benchmark_log_path, "r") as benchmark_log:
                lines = benchmark_log.readlines()
                
            for line in lines:
                if "initial accuracy:" in line:
                    clean_acc = line[line.find(":") + 2 : -2]
                elif "robust accuracy:" in line:
                    robust_acc = line[line.find(":") + 2 : -2]
        else:
            clean_acc = {
                "hso_normal_training_robustbench_Wong2020Fast_imagenet1k_13.onnx": 51.74,
                "hso_normal_training_robustbench_Salman2020Do_R18_imagenet1k_13.onnx": 42.95,
                "hso_normal_training_robustbench_Salman2020Do_50_2_imagenet1k_13.onnx": 59.71 
            }[file]
            print("No benchmark for", file)
        
    if file.startswith("timm_"):
        architecture = {
            "timm_resnet18_imagenet1k_13.onnx": "ResNet-18",
            "timm_resnet50_imagenet1k_13.onnx": "ResNet-50",
            "timm_wide_resnet50_2_imagenet1k_13.onnx": "WideResNet-50-2"
        }[file]
        
        clean_acc = {
            "timm_resnet18_imagenet1k_13.onnx": 69.74,
            "timm_resnet50_imagenet1k_13.onnx": 80.37,
            "timm_wide_resnet50_2_imagenet1k_13.onnx": 81.45
        }[file] 
    
    # TODO add robust accuracy for timm models and our imagenet models (or just go for 0)
    
    entry_dict = {
        "Name": file.replace(".onnx", ""),
        "Paper": paper_link,
        "Tracer Warning": None,
        "Selection": None,
        "Hide": None,
        "Robust": robust,
        "Clean Accuracy": clean_acc,
        "Robust Accuracy": robust_acc,
        "Network": paper_id,
        "Backbone": architecture,
        "Framework": "pytorch",
        "Pretraining-Dataset": additional_data,
        "Training-Dataset": dataset,
        "Visual Category": "natural",
        "Visual Category_micro": None,
        "Precision": "fp32",
        "Conv Weight Initializer": None,
        "Optimizer": None,
        "Task": "Classification",
        "Input Shape": None,
        "Accessible": None
    }
    entries.append(entry_dict)
    
df = pd.DataFrame(entries)
df.set_index("Name").to_csv("meta.csv")

No benchmark for hso_normal_training_robustbench_Salman2020Do_50_2_imagenet1k_13.onnx
No benchmark for hso_normal_training_robustbench_Salman2020Do_R18_imagenet1k_13.onnx
No benchmark for hso_normal_training_robustbench_Wong2020Fast_imagenet1k_13.onnx


### Sanity Checks

In [86]:
df[(df["Clean Accuracy"].astype(float) < 1)]

,Name,Paper,Tracer Warning,Selection,Hide,Robust,Clean Accuracy,Robust Accuracy,Network,Backbone,...,Pretraining-Dataset,Training-Dataset,Visual Category,Visual Category_micro,Precision,Conv Weight Initializer,Optimizer,Task,Input Shape,Accessible


In [87]:
df[(df.Robust) & (df["Robust Accuracy"].astype(float) < 1)]

,Name,Paper,Tracer Warning,Selection,Hide,Robust,Clean Accuracy,Robust Accuracy,Network,Backbone,...,Pretraining-Dataset,Training-Dataset,Visual Category,Visual Category_micro,Precision,Conv Weight Initializer,Optimizer,Task,Input Shape,Accessible


In [88]:
df[(~df.Robust) & (df["Robust Accuracy"].astype(float) > 0)]

,Name,Paper,Tracer Warning,Selection,Hide,Robust,Clean Accuracy,Robust Accuracy,Network,Backbone,...,Pretraining-Dataset,Training-Dataset,Visual Category,Visual Category_micro,Precision,Conv Weight Initializer,Optimizer,Task,Input Shape,Accessible
